# Generating a Lesson Plan Aligned with the BNCC Using the Bode Language Model

## Importing Modules

In [ ]:
!pip install unidecode
!pip install transformers
!pip install einops accelerate bitsandbytes
!pip install sentence_transformers
!pip install git+https://github.com/huggingface/peft.git

import os
import json
import time
import torch
import pandas as pd
from numba import cuda
from google.colab import drive
from unidecode import unidecode
from huggingface_hub import login
from google.colab import userdata
from peft import PeftModel, PeftConfig
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig

torch.cuda.empty_cache()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

## Utils

### Helper

In [ ]:
class Helper:

    def __init__(self):
      pass


    def expand_lines(self, df):
      """Expande linhas onde ano_aj contém múltiplos anos separados por vírgula"""

      grade_col = "ano_aj"    # Nome da coluna que contém os anos
      new_lines = []          # Lista para armazenar as linhas expandidas

      # Iterando sobre cada linha do df
      for _, row in df.iterrows():
          years = str(row[grade_col]).split(";")
          years = [grade.strip() for grade in years]

          if len(years) == 1:
            new_lines.append(row)
          else:
            for grade in years:
              new_line = row.copy()
              new_line[grade_col] = grade
              new_lines.append(new_line)

      return pd.DataFrame(new_lines)


    def get_value_or_empty(self, row, column_name):
      """Retorna o valor da coluna ou string vazia se for NaN"""
      return row[column_name] if pd.notna(row.get(column_name)) else ""


    def bncc_to_json(self, csv_path):
      """Converte a BNCC em JSON estruturado"""

      # Carregando o arquivo e replicando linhas
      df = pd.read_csv(csv_path, encoding='utf-8', delimiter=',')
      # df_expanded = self.expand_lines(df)

      # Agrupando por area, etapa_ensino (EF ou EM), ano e componente
      grouped = df.groupby(['area', 'etapa_ensino', 'ano_aj', 'componente'])

      # Criando o JSON
      bncc_json = {"bncc": []}

      for (area, etapa, ano, componente), group in grouped:
        entry = {
          "area": area,
          "etapa_ensino": etapa,
          "ano": ano,
          "componente": componente,
          "habilidades": []
        }

        for _, row in group.iterrows():
          habilidade = {
            "codigo": row['codigo_habilidade'],
            "competencia_especifica": self.get_value_or_empty(row, 'competencia_especifica'),
            "descricao": self.get_value_or_empty(row, 'habilidade')
          }
          entry["habilidades"].append(habilidade)

        bncc_json["bncc"].append(entry)

      # Salvando o JSON
      project_root = ""
      with open("/content/drive/MyDrive/Mestrado/Disciplinas/Inteligencia_Artificial/Codigo_Artigo/geracao-planos-bncc-colab/data/processed/bncc_data.json", 'w', encoding='utf-8') as f:
        json.dump(bncc_json, f, ensure_ascii=False, indent=2)

      return bncc_json

## Models

### SBERT

In [ ]:
class SBERT:
    def __init__(self, prompt, bncc_json_path):
        self.model = SentenceTransformer("sentence-transformers/msmarco-distilbert-cos-v5") # https://sbert.net/docs/pretrained-models/msmarco-v5.html
        self.prompt = prompt['tema']

        with open(bncc_json_path, 'r', encoding='utf-8') as file:
            self.bncc_json = json.load(file)

        # Pré-processamento
        self.bncc_sentences = self._get_bncc_sentences()
        self.bncc_skills = self._get_bncc_skills()
        self.prompt_embedding = self.model.encode(self.prompt, convert_to_tensor=True)
        self.bncc_embeddings = self.model.encode(self.bncc_sentences, convert_to_tensor=True)


    def _get_bncc_sentences(self):
        """Extrai descrições de habilidades da BNCC"""
        return [habilidade['descricao']
                for item in self.bncc_json["bncc"]
                for habilidade in item["habilidades"]]


    def _get_bncc_skills(self):
        """Extrai as competências específicas da BNCC"""
        return [competencia['competencia_especifica']
                for item in self.bncc_json["bncc"]
                for competencia in item["habilidades"]]


    def compute_similarities(self):
        """
        Calcula similaridades entre o prompt e cada embedding da BNCC.
        """
        self.similarities = util.cos_sim(self.prompt_embedding, self.bncc_embeddings)[0].cpu().tolist()

        return self.similarities


    def get_results(self):
        """
        Retorna as habilidades da BNCC mais similares ao prompt, com ou sem filtragem por threshold.
        """
        results = []
        scores = self.compute_similarities()

        # Verificação de consistência
        assert len(scores) == len(self.bncc_skills) == len(self.bncc_sentences), "Dimensões incompatíveis"

        for i, score_tensor in enumerate(scores):
            score = score_tensor.item() if hasattr(score_tensor, 'item') else float(score_tensor)

            results.append({
                'id': i,
                'score': score,
                'competencia_especifica': self.bncc_skills[i],
                'habilidade': self.bncc_sentences[i]
            })

        return sorted(results, key=lambda x: x['score'], reverse=True)


    def print_results(self):
        """Imprime os resultados formatados"""
        results = self.get_results()
        if not results:
            print(f"Nenhuma correspondência encontrada")
            return

        print(f"\nHabilidades da BNCC relacionadas a '{self.prompt}':")
        for item in results:
            print(f"\n[{item['id']}] Score: {item['score']:.3f}")
            print(f"Competência específica: {item['competencia_especifica']}")
            print(f"Habilidade: {item['habilidade']}")

### Bode

In [ ]:
class Bode:
    def __init__(self, token):
        self.token = token
        self._setup_environment()
        self._load_model()

        self.generation_config = GenerationConfig(
            temperature=0.7,
            max_new_tokens=800,
            top_p=0.85,
            do_sample=True
        )

        # Lista atualizada de exemplos de Few-shot Prompting
        self.examples = [
            # http://portaldoprofessor.mec.gov.br/fichaTecnicaAula.html?aula=49142
            {
                "input": "Tema: função conativa da linguagem e os textos publicitários. Competência específica: Compreender o funcionamento das diferentes linguagens e práticas (artísticas, corporais e verbais) e mobilizar esses conhecimentos na recepção e produção de discursos nos diferentes campos de atuação social e nas diversas mídias, para ampliar as formas de participação social, o entendimento e as possibilidades de explicação e interpretação crítica da realidade e para continuar aprendendo. Habilidade: Analisar o funcionamento das linguagens, para interpretar e produzir criticamente discursos em textos de diversas semioses.",
                "output": "Plano de Aula:\n- Objetivo:\n  1. Perceber a relação entre os textos publicitários e a função conativa da linguagem;\n  2. Analisar textos publicitários, identificando estratégias discursivas que expressam a função conativa;\n  3. Produzir textos publicitários que priorizem a função conativa da linguagem, utilizando recursos verbais e não verbais.\n\n- Conteúdo:\n  1. Funções da linguagem (ênfase na função conativa);\n  2. Elementos da comunicação;\n  3. Estratégias discursivas da publicidade;\n  4. Produção textual com foco persuasivo.\n\n- Recursos didáticos:\n  1. Quadro e giz;\n  2. Projeção de textos publicitários via data show ou distribuição de handouts;\n  3. Imagens de anúncios publicitários diversos;\n  4. Folhas em branco e materiais para produção de textos publicitários (lápis de cor, canetas etc.).\n\n- Metodologia:\n  1. Revisão das funções da linguagem com ênfase na função conativa;\n  2. Exposição de exemplos de textos publicitários com foco na função conativa e análise coletiva com base em três perguntas: qual a intenção discursiva, como ocorre a persuasão (direta ou indireta), e quais recursos linguísticos são utilizados;\n  3. Formação de duplas ou trios para análise de novos textos publicitários com as mesmas perguntas orientadoras;\n  4. Produção, em grupo, de dois textos publicitários sobre o mesmo produto: um direto e outro indireto, ambos com intenção persuasiva;\n  5. Apresentação das produções para a turma, que votará na versão mais persuasiva.\n\n- Avaliação:\n  1. Participação e interesse dos alunos nas atividades propostas;\n  2. Capacidade de análise crítica dos textos publicitários quanto à função conativa;\n  3. Produção de textos publicitários eficazes do ponto de vista persuasivo, com uso adequado de recursos verbais e não verbais."
            }
            # https://cursocompletodepedagogia.com/plano-de-aula-de-portugues-para-1-ano-do-ensino-medio/
            ,{
                "input": "Tema: aula sobre contos. Competência específica: Compreender o funcionamento das diferentes linguagens e práticas (artísticas, corporais e verbais) e mobilizar esses conhecimentos na recepção e produção de discursos nos diferentes campos de atuação social e nas diversas mídias, para ampliar as formas de participação social, o entendimento e as possibilidades de explicação e interpretação crítica da realidade e para continuar aprendendo. Habilidade: Estabelecer relações entre as partes do texto, tanto na produção como na leitura/escuta, considerando a construção composicional e o estilo do gênero, usando/reconhecendo adequadamente elementos e recursos coesivos diversos que contribuam para a coerência, a continuidade do texto e sua progressão temática, e organizando informações, tendo em vista as condições de produção e as relações lógico-discursivas envolvidas (causa/efeito ou consequência; tese/argumentos; problema/solução; definição/exemplos etc.).",
                "output": "Plano de Aula:\n- Objetivo:\n 1. Estudar o que é o conto e suas características;\n 2. Compreender os diferentes tipos de contos;\n 3. Explorar a estrutura e a história do conto;\n 4. Desenvolver a capacidade de análise e interpretação de textos narrativos.\n\n- Conteúdo:\n 1. Definição de conto;\n 2. Estrutura do conto (introdução, desenvolvimento, clímax e desfecho);\n 3. Tipos de contos (maravilhoso, realista, fantástico, entre outros);\n 4. Características linguísticas e estilísticas do conto;\n 5. Breve histórico sobre a origem e evolução do conto.\n\n- Recursos didáticos:\n 1. Projetor ou datashow;\n 2. Slides com textos e imagens ilustrativas sobre os tipos de contos;\n 3. Vídeos que abordem a história e características do conto;\n 4. Quadro branco e marcadores.\n\n- Metodologia:\n 1. Início da aula com uma explanação sobre o conceito de conto e sua importância na literatura;\n 2. Apresentação de vídeos e slides que exemplifiquem os diferentes tipos de contos e suas características;\n 3. Discussão em sala sobre as influências históricas e culturais no desenvolvimento do conto;\n 4. Formação de grupos para realização de uma pesquisa sobre os tipos de contos e suas características, com posterior apresentação dos resultados.\n\n- Avaliação:\n 1. Observação da participação e envolvimento dos alunos durante as atividades;\n 2. Avaliação das apresentações em grupo, considerando a compreensão do tema, a clareza na exposição e a capacidade de análise crítica;\n 3. Consideração das contribuições individuais nas discussões e atividades propostas."
            }
            #  https://planejamentosdeaula.com/plano-de-aula-funcoes-da-linguagem-ensino-medio-1o-ano/#gsc.tab=0
            ,{
                "input": "Tema: Funções da linguagem. Competência específica: Compreender e analisar os diferentes usos da linguagem em contextos diversos, reconhecendo suas funções e efeitos de sentido. Habilidade: Analisar o funcionamento das linguagens, para interpretar e produzir criticamente discursos em textos de diversas semioses (visuais, verbais, sonoras, gestuais).",
                "output": "Plano de Aula:\n- Objetivo:\n  1. Identificar e compreender as diferentes funções da linguagem;\n  2. Analisar textos diversos, reconhecendo as funções da linguagem presentes;\n  3. Produzir textos que evidenciem o uso consciente das funções da linguagem.\n\n- Conteúdo:\n  1. Funções da linguagem: referencial, emotiva, conativa, fática, metalinguística e poética;\n  2. Análise de textos diversos (propagandas, poemas, notícias, conversas);\n  3. Produção textual com foco nas funções da linguagem.\n\n- Recursos didáticos:\n  1. Quadro branco e marcadores;\n  2. Projetor multimídia;\n  3. Textos impressos de diferentes gêneros (poemas, propagandas, notícias);\n  4. Materiais de papelaria (papel, canetas, lápis).\n\n- Metodologia:\n  1. Apresentação teórica das funções da linguagem com exemplos práticos;\n  2. Leitura e análise de textos diversos, identificando as funções da linguagem presentes;\n  3. Atividades em grupo para discussão e aprofundamento do conteúdo;\n  4. Produção de textos individuais ou em grupo, aplicando as funções da linguagem estudadas;\n  5. Apresentação e discussão das produções realizadas.\n\n- Avaliação:\n  1. Participação nas atividades e discussões propostas;\n  2. Análise crítica e correta identificação das funções da linguagem nos textos analisados;\n  3. Produção textual coerente, evidenciando o uso adequado das funções da linguagem."
            }
        ]


    def _setup_environment(self):
      """Configuração inicial do ambiente CUDA"""
      torch.cuda.empty_cache()
      #cuda.select_device(0)
      #cuda.close()
      torch.backends.cuda.enable_flash_sdp(True)


    def _load_model(self):
        """Carregamento do modelo"""
        quantization_config = BitsAndBytesConfig(
            load_in_8bit=True,
            llm_int8_threshold=6.0,
            llm_int8_skip_modules=["lm_head"],
            bnb_4bit_compute_dtype=torch.bfloat16,
            llm_int8_enable_fp32_cpu_offload=True
        )

        llm_model = 'recogna-nlp/bode-7b-alpaca-pt-br'

        config = PeftConfig.from_pretrained(llm_model, token=self.token)

        self.model = AutoModelForCausalLM.from_pretrained(
            config.base_model_name_or_path,
            trust_remote_code=True,
            return_dict=True,
            device_map="auto",
            quantization_config=quantization_config,
            token=self.token
        )


        self.model = PeftModel.from_pretrained(
            self.model,
            llm_model,
            offload_folder="./offload"
        )

        self.tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, token=self.token)

        self.model.eval()
        torch.cuda.empty_cache()


    def _warmup_gpu(self):
        """Prepara a GPU para operação máxima"""
        if torch.cuda.is_available():
            dummy_input = self.tokenizer("Warmup", return_tensors="pt").to("cuda")
            with torch.no_grad():
                self.model.generate(**dummy_input, max_new_tokens=1)
            torch.cuda.empty_cache()


    def generate_prompt(self, instruction, input=None):
        prompt = (
            "Você é um pedagogo especialista em Língua Portuguesa para o Ensino Médio. "
            "Sua tarefa é gerar um plano de aula completo e alinhado à Base Nacional Comum Curricular, com base no tema, na competência específica e na habilidade fornecidos. "
            "Cada plano de aula deve conter as seguintes seções:\n"
            "- Objetivos\n- Conteúdo\n- Recursos didáticos\n- Metodologia\n- Avaliação\n\n"
            "Utilize a competência e a habilidade para guiar os objetivos, os métodos e as formas de avaliação. "
            "Baseie-se na estrutura e estilo dos exemplos abaixo, mantendo consistência pedagógica, clareza textual e adequação à realidade da sala de aula."
        )

        # Adiciona exemplos ao prompt
        for example in self.examples:
            prompt += f"### Instrução:\n{example['input']}\n\n### Resposta:\n{example['output']}\n"
            prompt += "\n" + "-"*50 + "\n"

        # Adiciona a tarefa atual
        prompt += f"### Instrução:\n{instruction}\n"
        if input:
            prompt += f"\n### Entrada:\n{input}\n"

        prompt += "\n### Resposta:"
        return prompt


    def evaluate(self, instruction, input=None):
        start_preprocess = time.time()

        prompt = self.generate_prompt(instruction, input)
        inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")

        preprocess_time = time.time() - start_preprocess

        start_generation = time.time()

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                generation_config=self.generation_config
            )

        generation_time = time.time() - start_generation

        output = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        print(f"\nTempo de pré-processamento: {preprocess_time:.3f}s")
        print(f"Tempo de geração: {generation_time:.3f}s")
        print(f"Tempo total: {preprocess_time + generation_time:.3f}s")
        print(f"\nResposta:\n{output.split('### Resposta:')[-1].strip()}")

        return output.split('### Resposta:')[-1].strip()

## Notebook

### Converting BNCC to JSON

In [ ]:
helper = Helper()
csv_path = "/content/drive/MyDrive/Mestrado/Disciplinas/Inteligencia_Artificial/Codigo_Artigo/geracao-planos-bncc-colab/data/raw/bncc_data.csv"

# Verifica se o arquivo existe
if not os.path.exists(csv_path):
    print(f"Arquivo não encontrado: {csv_path}")
else:
    bncc_json = helper.bncc_to_json(csv_path)

### Defining the input prompt

In [ ]:
prompt = {
    "ano": 2,
    "area": "linguagens e suas tecnologias",
    "tema": "função conativa da linguagem e os textos publicitários"
}

### Applying SBERT

#### Loading Model

In [ ]:
bncc_path = "/content/drive/MyDrive/Mestrado/Disciplinas/Inteligencia_Artificial/Codigo_Artigo/geracao-planos-bncc-colab/data/processed/bncc_data.json"

sbert = SBERT(prompt, bncc_path)

#### Computing Similarities

In [ ]:
# Opção 1: Obter similaridades brutas
similarities = sbert.compute_similarities()

# Opção 2: Obter resultados processados
results = sbert.get_results()

# Opção 3: Imprimir resultados formatados
sbert.print_results()

first_result = results[0]['habilidade']
skills = results[0]['competencia_especifica']
# print(skills)
tema = "Tema: " + prompt['tema']


Habilidades da BNCC relacionadas a 'função conativa da linguagem e os textos publicitários':

[11] Score: 0.576
Competência específica: Compreender o funcionamento das diferentes linguagens e práticas culturais (artísticas, corporais e verbais) e mobilizar esses conhecimentos na recepção e produção de discursos nos diferentes campos de atuação social e nas diversas mídias, para ampliar as formas de participação social, o entendimento e as possibilidades de explicação e interpretação crítica da realidade e para continuar aprendendo. Mobilizar práticas de linguagem no universo digital, considerando as dimensões técnicas, críticas, criativas, éticas e estéticas, para expandir as formas de produzir sentidos, de engajar-se em práticas autorais e coletivas, e de aprender a aprender nos campos da ciência, cultura,trabalho, informação e vida pessoal e coletiva.
Habilidade: Selecionar informações, dados e argumentos em fontes confiáveis, impressas e digitais, e utilizá-los de forma referenciad

### Generating text with Bode

#### Loading Model

In [ ]:
torch.cuda.empty_cache()
token = userdata.get('HF_TOKEN')
bode = Bode(token)

print(torch.cuda.get_device_name(0))
print(f"Memória Total: {round(torch.cuda.get_device_properties(0).total_memory / 1e9, 2)} GB")
print(f"Memória Reservada: {round(torch.cuda.memory_reserved(0) / 1e9, 2)} GB")
print(f"Memória Alocada: {round(torch.cuda.memory_allocated(0) / 1e9, 2)} GB")

adapter_config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Tesla T4
Memória Total: 15.83 GB
Memória Reservada: 8.08 GB
Memória Alocada: 7.3 GB


#### Generating Lesson Plan

In [ ]:
instruction = f"Tema: {tema}. Competência específica: {skills}. Habilidade: {first_result}"

output = bode.evaluate(instruction)


Tempo de pré-processamento: 0.009s
Tempo de geração: 70.839s
Tempo total: 70.848s

Resposta:
Plano de Aula:
- Objetivo:
  1. Compreender a função conativa da linguagem e sua aplicação nos textos publicitários;
  2. Analisar e comparar os diferentes tipos de textos publicitários, identificando as diferentes estratégias discursivas;
  3. Produzir textos publicitários, utilizando as diferentes estratégias discursivas, para acompanhar a evolução da linguagem e da comunicação.

- Conteúdo:
  1. Funções da linguagem conativa;
  2. Análise de diferentes tipos de textos publicitários;
  3. Produção de textos publicitários, considerando a função conativa.

- Recursos didáticos:
  1. Projetor multimídia;
  2. Textos impressos de diferentes gêneros (propagandas, anúncios, notícias);
  3. Materiais de papelaria (papel, canetas, lápis).

- Metodologia:
  1. Apresentação teórica das funções da linguagem conativa;
  2. Leitura e análise de textos publicitários, identificando as diferentes estratégia

### Evaluating

In [ ]:
from collections import Counter
import math

def calculate_ttr(text):
    words = text.split()
    unique_words = set(words)

    ttr = len(unique_words) / len(words) if words else 0

    print(f"Taxa de Tipo-Token: {ttr:.2%}")

    return ttr

def calculate_entropy(text):
    words = text.split()
    if not words:
        return 0.0

    freq = Counter(words)
    total = len(words)
    unique_count = len(freq)

    max_entropy = math.log2(unique_count) if unique_count > 0 else 0.0

    ent = 0.0
    for count in freq.values():
        p = count / total
        ent -= p * math.log2(p) if p > 0 else 0

    print(f"Entropia máxima teórica: {max_entropy:.4f} bits")
    print(f"Entropia observada: {ent:.4f} bits")
    print(f"Razão entropia/máxima: {ent/max_entropy:.2%}" if max_entropy > 0 else "N/A")

    return ent

In [ ]:
ttr = calculate_ttr(output)

Taxa de Tipo-Token: 50.82%


In [ ]:
entropy = calculate_entropy(output)

Entropia máxima teórica: 6.5392 bits
Entropia observada: 6.0520 bits
Razão entropia/máxima: 92.55%
